# FRUIT CLASSIFICATION

In [ ]:
!pip install -U albumentations
!pip install opencv-python==4.5.4.60
!pip install timm==0.6.2.dev0

^C
^C


In [9]:
import pandas as pd
import pickle
import os
import csv

### Creating the Train Dataset

In [7]:
import os
import csv
# from pickle import Image
# Path to folder containing the subfolders
path = '../input/ds_frutas_am/train/'

# Create the csv file
with open('train.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['filename', 'class'])

    # Loop through each folder
    for folder in os.listdir(path):
        # Loop through each image in the folder
        for image in os.listdir(path + folder):
            # Write the filename and class to the csv file
            writer.writerow([image, folder])

### Creating the test File

In [16]:
# Create the csv file
path = '../input/ds_frutas_am/test/'
with open('test.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['filename', 'class'])

    # Loop through each folder
    for folder in os.listdir(path):
        # Loop through each image in the folder
        for image in os.listdir(path + folder):
            # Write the filename and class to the csv file
            writer.writerow([image, folder])

In [17]:
train = pd.read_csv('/kaggle/working/train.csv')
test = pd.read_csv('/kaggle/working/test.csv')
train.shape, test.shape

((90, 2), (30, 2))

### Lets go modelling

In [ ]:
!pip install -U albumentations
!pip install opencv-python==4.5.4.60
!pip install timm==0.6.2.dev0

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import os
import albumentations
from albumentations.pytorch import ToTensorV2
import pandas as pd
import numpy as np
import gc
import timm
from fastai.vision.all import *

In [ ]:
def random_seed(seed_value, use_cuda): 
    np.random.seed(seed_value)
 #cpu vars
    torch.manual_seed(seed_value) 
# cpu  vars
    random.seed(seed_value)
 # Python 
    if use_cuda: 
        torch.cuda.manual_seed(seed_value) 
        torch.cuda.manual_seed_all(seed_value) 
# gpu vars
        torch.backends.cudnn.deterministic = True 
 #needed
        torch.backends.cudnn.benchmark = False 
#Remember to use num_workers=0 when creating the DataBunch.

In [ ]:
random_seed(2022,True)

In [ ]:
path = '/content/gdrive/MyDrive/sign_prediction/'

df = pd.read_csv(path + 'Train.csv')


df.img_IDS    = ["/content/Images/content/ModImages/"+i+".jpg" for i in df.img_IDS]

df.head()

In [ ]:
df.Label.value_counts()

In [ ]:
class AlbumentationsTransform (RandTransform):
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [ ]:
def get_train_aug(): return albumentations.Compose([
            albumentations.Resize(224, 224), #Extra tip, use size that's suitable for the efficentNetwork you are using.
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.ColorJitter(),
            albumentations.Blur(),
            ], p=1.)



def get_valid_aug(): return albumentations.Compose([
            albumentations.Resize(224, 224),
            ], p=1.0)

item_tfms = AlbumentationsTransform(get_train_aug(), get_valid_aug())
batch_tfms = [Normalize.from_stats(*imagenet_stats)]

In [ ]:
index_splitter = IndexSplitter(train_idxs = df_train.index, valid_idxs = df_test.index)

In [ ]:
block = DataBlock(blocks=(ImageBlock, CategoryBlock), 
    splitter=TrainTestSplitter(0.05, stratify=df["Label"]),
    get_x = ColReader(0),
    item_tfms=item_tfms, 
    get_y=ColReader(1),
    batch_tfms = batch_tfms
    )


In [ ]:
dls = block.dataloaders(df, bs=8)
dls.show_batch(figsize=(12,12))

In [ ]:
from fastai.metrics import accuracy
timm.list_models('vit*')

In [ ]:
learn = vision_learner(dls, 'vit_large_patch16_224',  metrics=[accuracy], cbs=[GradientAccumulation(),SaveModelCallback(),EarlyStoppingCallback(patience=3)])
learn.to_fp16()

In [ ]:
# del variables
gc.collect()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(15,0.001737800776027143)

In [ ]:
learn.to_fp32()
pickle.dump(learn, open('model.pkl', 'wb'))